# Web安全之机器学习
## 第5章 K近邻算法
### 5.6 示例：使用K近邻算法检测WebShell

#### 1.数据收集和数据清洗
使用的数据集为ADFA-LD主机级入侵检测系统的数据集，记录了系统的调用数据。数据分为三类：“Train_data”、“Attack_data”、“Validation_data”。

我们将使用“Train_data”、“Attack_data”用来训练，其中“Train_data”的标签为正常，“Attack_data”的标签为异常。

每一个样本里，为一系列的系统调用的编号组成的调用序列。

首先（1）加载ADFA-LD中正常样本的数据。

In [1]:
import os
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [2]:
def get_noranl_data(rootdir):
    file_list = os.listdir(rootdir)
    x = []
    y =[]
    for file in file_list:
        file_path = os.path.join(rootdir,file)
        if os.path.isfile(file_path):
            with open(file_path,'r') as f:
                x.append(f.read())
                y.append(0)
    return x,y

（2）遍历指定文件下的目录

In [3]:
def dirlist(rootdir):
    all_file = []
    path_list = os.listdir(rootdir)
    for path in path_list:
        file_path = os.path.join(rootdir,path)
        if os.path.isdir(file_path):
            file_path += '/'
            all_file += dirlist(file_path)
        else:
            all_file.append(file_path)
    return all_file

（3）从攻击数据集中筛选出和WebShell相关的数据

In [4]:
def get_attack_data(rootdir):
    all_file = dirlist(rootdir)
    x = []
    y =[]
    for file in all_file:
        pattern = re.compile("../data/ADFA-LD/Attack_Data_Master/Web_Shell_\d+/UAD-W*")
        if re.match(pattern,file):
            with open(file,'r') as f:
                x.append(f.read())
                y.append(1)
    return x,y

#### 2.特征化
现在我们获得的每个样本，均为一系列的系统调用编号，有长有短，这里我们可以使用词集模型将其特征化。

In [5]:
def get_feature():
    x1,y1 = get_noranl_data("../data/ADFA-LD/Training_Data_Master/")
    x2,y2 = get_attack_data("../data/ADFA-LD/Attack_Data_Master/")
    x = x1+x2
    Y = y1+y2

    vectorizer = CountVectorizer(min_df=1)  ###将词集向量化
    X = vectorizer.fit_transform(x)
    return X,Y


#### 3.训练样本和交叉验证

In [6]:
def main():
    X,Y = get_feature()
    neigh = KNeighborsClassifier(n_neighbors=3)
    score = cross_val_score(neigh,X,Y,cv=10,n_jobs=1)
    print(score)
main()

[ 0.95833333  0.94791667  0.97916667  0.96842105  0.96842105  0.84210526
  0.97894737  0.98947368  0.9787234   0.9787234 ]
